<a href="https://colab.research.google.com/github/rajesh0305/M_Tech_Project/blob/main/whdld_dppnet_Copy3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import sys
import random
import warnings

import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
  except RuntimeError as e:
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
color_dict = {
    0: [128, 128, 128],     # gray = bare soil
    1: [255, 0, 0],         # red = building
    2: [192, 192, 0],       # olive = pavement
    3: [255, 255, 0],       # yellow = road
    4: [0, 255, 0],         # green = vegetation
    5: [0, 0, 255],         # blue = water
}

def rgb_to_onehot(rgb_arr,color_dict=color_dict):
    num_classes = len(color_dict)
    shape = rgb_arr.shape[:2]+(num_classes,)
    arr = np.zeros(shape,dtype = np.int8)
    for i,cls in enumerate(color_dict):
        arr[:,:,i] = np.all(rgb_arr.reshape((-1,3))==color_dict[i],axis=1).reshape(shape[:2])
        arr = np.array(arr)
    mask = np.argmax(arr,axis=-1)
    return mask

def onehot_to_rgb(onehot,color_dict = color_dict):
    output = np.zeros(onehot.shape[:2]+(3,))
    for k in color_dict.keys():
        output[onehot==k]=color_dict[k]
    return output

def get_dataset(X_dir, y_dir):

    num_samples = len(X_dir)
    X = []
    y = []

    for i in range(num_samples):
        X.append(plt.imread(X_dir[i]))
        y.append(rgb_to_onehot(((plt.imread(y_dir[i]))[:,:,0:3])*255))

    y = np.reshape(y, (-1, 256, 256, 1))
    return np.array(X), np.array(y)

In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np

X_train = glob.glob("I:/WHDLD/train/X/*.jpg")
y_train = glob.glob("I:/WHDLD/train/y/*.png")

X_test = glob.glob("I:/WHDLD/test/X/*.jpg")
y_test = glob.glob("I:/WHDLD/test/y/*.png")

X_train.sort()
y_train.sort()

X_test.sort()
y_test.sort()

print(len(X_train), len(y_train), len(X_test), len(y_test))
X_train, y_train = get_dataset(X_train, y_train)
X_test, y_test = get_dataset(X_test, y_test)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

3703 3703 414 414
(3703, 256, 256, 3) (3703, 256, 256, 1) (414, 256, 256, 3) (414, 256, 256, 1)


In [ ]:
np.save('I:/WHDLD/X_train.npy',X_train)
np.save('I:/WHDLD/y_train.npy',y_train)
np.save('I:/WHDLD/X_test.npy',X_test)
np.save('I:/WHDLD/y_test.npy',y_test)

In [ ]:
X_train = np.load('I:/WHDLD/X_train.npy')
y_train = np.load('I:/WHDLD/y_train.npy')
X_test = np.load('I:/WHDLD/X_test.npy')
y_test = np.load('I:/WHDLD/y_test.npy')

In [ ]:
import numpy as np
from keras.backend import int_shape
from keras.models import Model
from keras.layers import Conv2D,Conv2DTranspose, Conv3D, MaxPooling2D, MaxPooling3D, UpSampling2D, Add, BatchNormalization, Input, Activation, Lambda, Concatenate
from tensorflow.keras import regularizers


def residual_separable(inp, n_filters, dropout=0.3, dilation=1, l2=None, name="down"):
    x = tf.keras.layers.SeparableConv2D(n_filters, (3, 3), strides=1, padding='same', activation=None,
                        dilation_rate=dilation, use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform',
                        pointwise_regularizer=regularizers.l2(0.00004))(inp)
    x = BatchNormalization()(x)
    x = Dropout(rate=dropout)(x)
    if inp.shape[3] == x.shape[3]:
        x = Add()([x, inp])
    x = Activation('relu')(x)
    return x

def residual_separable_multi(inp, n_filters, dropout=0.3, dilation=1, l2=None, name="down"):
    x = tf.keras.layers.DepthwiseConv2D(3, strides=(1, 1), depth_multiplier=1, padding='same', use_bias=False)(inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x2 = tf.keras.layers.DepthwiseConv2D(3, strides=(1, 1), depth_multiplier=1, dilation_rate= (dilation, dilation), padding='same', use_bias=False)(inp)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)

    x +=x2

    x = Conv2D(n_filters, 1, strides=1, padding='same', activation=None,
                         dilation_rate=1, use_bias=False, kernel_regularizer=regularizers.l2(0.00004))(x)
    x = BatchNormalization()(x)

    x = Dropout(rate=dropout)(x)

    if inp.shape[3] == x.shape[3]:
        x = Add()([x, inp])

    x = Activation('relu')(x)
    return x


def encoder_module(inp, n_filters, dropout=0.3, dilation=[1,1], l2=None, name="down"):
    x = residual_separable(inp, n_filters, dropout=dropout, dilation=dilation[0], l2=l2, name=name)
    x = residual_separable(x, n_filters, dropout=dropout, dilation=dilation[1], l2=l2, name=name)
    return x

def encoder_module_multi(inp, n_filters, dropout=0.3, dilation=[1,1], l2=None, name="down"):
    x = residual_separable_multi(inp, n_filters, dropout=dropout, dilation=dilation[0], l2=l2, name=name)
    x = residual_separable_multi(x, n_filters, dropout=dropout, dilation=dilation[1], l2=l2, name=name)
    return x

def upsample(x, n_filters, last=False, l2=None, name="down"):
    x = Conv2DTranspose(n_filters, 3, strides=2, padding='same',use_bias=True,
                                   activation=None,
                                   kernel_regularizer=regularizers.l2(0.00004))(x)
    if not last:
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    return x

def downsample(inp, n_filters_in, n_filters_out, bn=False, use_relu=False, l2=None, name="down"):

    if n_filters_in < n_filters_out:
        filters_conv = n_filters_out - n_filters_in
    else:
        filters_conv = n_filters_out

    x = Conv2D(filters_conv, 3, strides=2, padding='same', activation=None,
                         dilation_rate=1, use_bias=False, kernel_regularizer=regularizers.l2(0.00004))(inp)

    if n_filters_in < n_filters_out:
        y =  MaxPooling2D(pool_size=(2, 2), strides=(2,2))(inp)
        x =  concatenate([x,y], axis = -1)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def DPP(x,f):
  x1 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same', activation='relu',
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x)
  x2 = tf.keras.layers.SeparableConv2D(f, (3, 3), strides = 2, padding='same', activation='relu',
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x)
  x3 = tf.keras.layers.SeparableConv2D(f, (3, 3), strides = 4, padding='same', activation='relu',
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x)
  x1 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same', activation='relu',
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x1)
  x2 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same', activation='relu',
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x2)
  x3 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same', activation='relu',
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x3)
  x1 = Conv2D(f,(1,1))(x1)
  x2 = Conv2DTranspose(f, 3, strides=2, padding='same',use_bias=True, activation='relu')(x2)
  x3 = Conv2DTranspose(f, 3, strides=4, padding='same',use_bias=True, activation='relu')(x3)
  x = concatenate([x3,x2,x1], axis=3)
  x1 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same',dilation_rate = (1,1), activation='relu',
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x)
  x2 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same',dilation_rate = (2,2), activation='relu',
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x)
  x3 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same',dilation_rate = (4,4), activation='relu',
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x)
  x4 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same',dilation_rate = (8,8), activation='relu',
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x)
  x = concatenate([x1,x2,x3,x4], axis=3)
  x = Conv2D(f,(1,1))(x)
  return x

In [ ]:
import numpy as np
from keras.backend import int_shape
from keras.models import Model
from keras.layers import *
from tensorflow.keras import regularizers
import tensorflow as tf

def feature_extractor(x,f,dropout=0.25, dilation=[1,1,1,1], l2=None, name="down"):
  x1_1 = residual_separable_multi(x, f, dropout=dropout, dilation=1, l2=l2, name=name)
  x1_2 = residual_separable_multi(x, f//4, dropout=dropout, dilation=dilation[0], l2=l2, name=name)
  x2 = concatenate([x,x1_2])
  x2_1 = residual_separable_multi(x2, f, dropout=dropout, dilation=1, l2=l2, name=name)
  x2_2 = residual_separable_multi(x2_1, f//4, dropout=dropout, dilation=dilation[1], l2=l2, name=name)
  x3 = concatenate([x,x1_2,x2_2])
  x3_1 = residual_separable_multi(x3, f, dropout=dropout, dilation=1, l2=l2, name=name)
  x3_2 = residual_separable_multi(x3_1, f//4, dropout=dropout, dilation=dilation[2], l2=l2, name=name)
  x4 = concatenate([x,x1_2,x2_2,x3_2])
  x4_1 = residual_separable_multi(x4, f, dropout=dropout, dilation=1, l2=l2, name=name)
  x4_2 = residual_separable_multi(x4_1, f//4, dropout=dropout, dilation=dilation[3], l2=l2, name=name)
  xout = concatenate([x,x1_2,x2_2,x3_2,x4_2])
  return xout


In [ ]:
from tensorflow.keras.layers import BatchNormalization, Activation, AveragePooling2D, DepthwiseConv2D, Conv2DTranspose
import tensorflow.keras.backend as K
from keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Activation,Reshape, Add, Multiply, DepthwiseConv2D, BatchNormalization, Concatenate, Conv2D, Dense,Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D, Input, Lambda,LeakyReLU, MaxPooling2D, Multiply, Permute, Reshape, UpSampling2D
import collections
import tensorflow as tf


def DPPNet(input_size,  n_classes, l2=None, upsampling=2):
    inputs = Input(input_size)
    d1 = downsample(inputs, 3, 16, l2=l2, name="d1")
    d2 = downsample(d1, n_filters_in=16, n_filters_out=64, l2=l2, name="d2")
    m1 = encoder_module(d2, n_filters=64, dilation=[1, 1], l2=l2, name="fres3", dropout=0.0)
    m2 = encoder_module(m1, n_filters=64, dilation=[1, 1], l2=l2, name="fres4", dropout=0.0)
    m3 = encoder_module(m2, n_filters=64, dilation=[1, 1], l2=l2, name="fres5", dropout=0.0)
    m4 = encoder_module(m3, n_filters=64, dilation=[1, 1], l2=l2, name="fres6", dropout=0.0)
    m5 = encoder_module(m4, n_filters=64, dilation=[1, 1], l2=l2, name="fres7", dropout=0.0)

    d3 = downsample(m5,  n_filters_in=64, n_filters_out=128, l2=l2, name="d8")
    m6 = feature_extractor(d3,128,dropout=0.25, dilation=[2,4,8,16], l2=l2, name="fres9")
    m7 = feature_extractor(m6,128,dropout=0.25, dilation=[1,2,8,16], l2=l2, name="fres10")
    up1 = upsample(m7, n_filters=64, l2=l2, name="up11")
    x = up1+d2
    up2 = upsample(x, n_filters=16, l2=l2, name="up16", last = True)
    x = concatenate([up2, d1], axis=3)
    x = DPP(x,32)
    x = Conv2D(6, (1, 1), activation='softmax')(x)
    if upsampling > 1:
      x = tf.keras.layers.experimental.preprocessing.Resizing(x.shape[1] * upsampling, x.shape[2] * upsampling)(x)


    model = Model(inputs, x)

    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

model = DPPNet((256,256,3), 6)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 13) 351         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 128, 128, 16) 0           conv2d[0][0]                     
                                                                 max_pooling2d[0][0]          

In [ ]:
from keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint('I:/WHDLD/mmini_whdld2.h5', monitor='loss', save_best_only=True)
callback=tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=4, callbacks = [model_checkpoint,callback] )

Epoch 1/100
926/926 [==============================] - 150s 154ms/step - loss: 1.2120 - accuracy: 0.5720
Epoch 2/100
926/926 [==============================] - 145s 157ms/step - loss: 0.8274 - accuracy: 0.7033
Epoch 3/100
926/926 [==============================] - 145s 157ms/step - loss: 0.7146 - accuracy: 0.7446
Epoch 4/100
926/926 [==============================] - 145s 157ms/step - loss: 0.6880 - accuracy: 0.7541
Epoch 5/100
926/926 [==============================] - 145s 156ms/step - loss: 0.6521 - accuracy: 0.7661
Epoch 6/100
926/926 [==============================] - 145s 156ms/step - loss: 0.6267 - accuracy: 0.7753
Epoch 7/100
926/926 [==============================] - 144s 156ms/step - loss: 0.6231 - accuracy: 0.7771
Epoch 8/100
926/926 [==============================] - 145s 156ms/step - loss: 0.6083 - accuracy: 0.7826
Epoch 9/100
926/926 [==============================] - 144s 156ms/step - loss: 0.5868 - accuracy: 0.7907
Epoch 10/100
926/926 [==============================] -

In [ ]:
model.save('I:/WHDLD/mminiwhdld2_model.h5')

y_pred = model.predict(X_test)

from tensorflow.keras.metrics import MeanIoU, Accuracy

meaniou = MeanIoU(6, name=None, dtype=None)
meaniou.update_state(y_test, y_pred.argmax(axis=3))
print(meaniou.result().numpy())
meaniou.reset_states()

acc = Accuracy()
acc.update_state(y_test, y_pred.argmax(axis=3))
print(acc.result().numpy())
acc.reset_states()

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test.ravel(), y_pred.argmax(axis=3).ravel())
print(cm)

from sklearn.metrics import classification_report
cr = classification_report(y_test.ravel(), y_pred.argmax(axis=3).ravel())
print(cr)

def get_dice(y_true, y_pred):

    intersection = np.sum(y_true * y_pred, axis=(0, 1, 2))
    union = np.sum(y_true**2, axis=(0, 1, 2)) + np.sum(y_pred**2, axis=(0, 1, 2))
    dc = 2 * intersection / union
    return dc

print(y_pred.shape, tf.one_hot(y_test.reshape((-1, 256, 256)), depth=6).numpy().shape)

dice_coeff = get_dice(tf.one_hot(y_test.reshape((-1,256, 256)), depth=6).numpy(), y_pred)
print(dice_coeff)
print(np.mean(dice_coeff))

from sklearn.metrics import cohen_kappa_score
k = cohen_kappa_score(y_test.ravel(), y_pred.argmax(axis=3).ravel())
print("k",k)

0.60920376
0.8102886
[[  643619    52248   257058    20334   165204    20437]
 [   24170  2839378   422343     9438   418638     3055]
 [  165119   643067  2412121   109308   621321     5991]
 [   26018    34357   239360   810385   134023     4072]
 [  224339   410782   683648    99986 11752730   146671]
 [    7927     5279     2872     2098   188068  3526440]]
              precision    recall  f1-score   support

           0       0.59      0.56      0.57   1158900
           1       0.71      0.76      0.74   3717022
           2       0.60      0.61      0.60   3956927
           3       0.77      0.65      0.70   1248215
           4       0.88      0.88      0.88  13318156
           5       0.95      0.94      0.95   3732684

    accuracy                           0.81  27131904
   macro avg       0.75      0.73      0.74  27131904
weighted avg       0.81      0.81      0.81  27131904

(414, 256, 256, 6) (414, 256, 256, 6)
[0.6250159 0.7889846 0.6634375 0.7337095 0.9251705 0.96

In [ ]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2_as_graph
#https://github.com/tokusumi/keras-flops/blob/master/notebooks/flops_calculation_tfkeras.ipynb
def get_flops(model, batch_size=None):
    if batch_size is None:
        batch_size = 1

    real_model = tf.function(model).get_concrete_function(tf.TensorSpec([batch_size] + model.inputs[0].shape[1:], model.inputs[0].dtype))
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(real_model)

    run_meta = tf.compat.v1.RunMetadata()
    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
    flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                            run_meta=run_meta, cmd='op', options=opts)
    return flops.total_float_ops

flops = get_flops(model,4)
flops

10131931136

In [ ]:
i = 18
test_img = model.predict(X_test[i,:,:,:].reshape((1, 256, 256, 3)))
test_img = np.argmax(test_img, axis=3)

# print(y_pred.shape, y_data[i, :, :].shape)

test_img = onehot_to_rgb(test_img.reshape((256, 256)))
#test_img_ = plt.imread(y_test[i])

plt.figure()
plt.imshow(test_img/255)

plt.figure()
plt.imshow((onehot_to_rgb(y_test[i,:,:].reshape((256, 256))))/255)

plt.figure()
plt.imshow(X_test[i, :, :, :])

In [ ]:
import time
t10 = time.time()
i = 51
test_img = model.predict(X_test[i,:,:,:].reshape((1, 256, 256, 3)))
#test_img = np.argmax(test_img, axis=3)
t11 = time.time()
time_forloop = t11 - t10
print(time_forloop)

2.8339028358459473
